# **IP4_Run1**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 8.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [ ]:
problem = """Imagine you own a small shop, and you're trying to manage your employees' working hours and sales targets efficiently.
You have three kinds of goals overall: sales goals, employee hours goals and overtime goals.

Sales Goals:
You have a target of selling 5500 units.
But sometimes, you might sell fewer (negative deviation) or more (positive deviation) than this target.
On average, your full-time employees will sell 5 records per hour and your part-time employees will sell 2 records per hour.

Employee Hours:
You have both full-time and part-time employees.
Your full-time employees are expected to work a total of 800 hours, while part-time employees are expected to work 320 hours.
Similarly, like sales, there could be deviations in these working hours.

Overtime:
Additionally, your employees might end up working overtime.
You have a target of 100 hours of overtime for full-time employees.
You are not concerned about limiting the overtime of part-time employees.
Again, you might deviate from this goal.

Each of these deviations has a different priority for you.
P1 to P4 are priority factors for each deviation.
Your most important goal is to achieve your sales goal.
Second to that is the goal to not exceed 100 overtime hours for full-time employees.
Then, you next priority is to fully utilize your full-time and part time employees.
Your last goal with the lowest priority, you want to keep general overtime hours for both kinds of employees as low as possible.
In addition to the general priorities of you goals, you are twice as concerned about fully utilizing the full-time employee hours.
Finally, overtime hours of part-time employees have a relative cost of three times that of full-time employees.

Now, the goal here is to minimize the overall impact of these deviations, considering their priorities.
This means finding the best balance between sales targets, employee hours, and overtime, while keeping in mind which deviations are more critical for your business."""

In [ ]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [ ]:
print(response1.choices[0].message.content)

Let's denote the variables needed for this problem:

1. S = Amount of units sold.
2. SF = Amount of units sold by full-time employees.
3. SP = Amount of units sold by part-time employees.
4. H = Total hours worked by all employees.
5. HF = Hours worked by full-time employees.
6. HP = Hours worked by part-time employees.
7. O = Total overtime hours.
8. OF = Overtime hours by full-time employees.
9. OP = Overtime Hours by part-time employees.

Deviation Variables: 

1. DS = Deviation from sales target (|S - 5500|).
2. DSF = Deviation in units sold by full-time employees (|SF - x|), where x is expected amount of units sold by full-time employees.
3. DSP = Deviation in units sold by part-time employees (|SP - y|), where y is expected amount of units sold by part-time employees. 
4. DH = Deviation from total hours target (|H - 1120|). 
5. DHF = Deviation from full-time employee hours target (|HF - 800|).
6. DHP = Deviation from part-time employee hours target (|HP - 320|). 
7. DO = Deviatio

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
print(response2.choices[0].message.content)

The objective function to minimize these deviations based on their priority factors would be:

Minimize Z = P1*DS + P2*DOF + P3*(2*DHF + DHP) + P4*(DO + 3*DOP)


In [ ]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
print(response3.choices[0].message.content)

The constraints for this mathematical optimization problem is:

1. Sales Constraints:
   - SF + SP = S 
   - SF = 5 * HF 
   - SP = 2 * HP 

2. Hours Constraints:
   - HF + HP = H 
   - HF <= 800 
   - HP <= 320 

3. Overtime Constraints:
   - OF + OP = O
   - OF <= 100 

4. Deviation Constraints: 
   
   For Sales:
   - DS >= S - 5500 
   - DS >= 5500 - S 

   For Full-Time Employees sales:
   - DSF >= SF - x 
   - DSF >= x - SF 

   For Part-Time Employees sales:
   - DSP >= SP - y 
   - DSP >= y - SP 

   For Overall hours:
   - DH >= H - 1120 
   - DH >= 1120 - H 

   For Full-Time Employees hours:
   - DHF >= HF - 800 
   - DHF >= 800 - HF

   For Part-Time Employees hours:
   - DHP >= HP - 320 
   - DHP >= 320 - HP 

   For Overall Overtime:
   - DO >= O - z 
   - DO >= z - O 

   For Full-Time Employees Overtime:
   - DOF >= OF - 100 
   - DOF >= 100 - OF 

   For Part-Time Employees Overtime:
   - DOP >= OP 
   - DOP >= -OP

Based on the problem statement there are no explicit 

In [ ]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
Markdown(response4.choices[0].message.content)

Below is the pyomo code for the problem:

```python
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Create a model
model = ConcreteModel()

# Define the decision variables
model.s = Var(domain=NonNegativeReals)
model.sf = Var(domain=NonNegativeReals)
model.sp = Var(domain=NonNegativeReals)
model.h = Var(domain=NonNegativeReals)
model.hf = Var(domain=NonNegativeReals)
model.hp = Var(domain=NonNegativeReals)
model.o = Var(domain=NonNegativeReals)
model.of = Var(domain=NonNegativeReals)
model.op = Var(domain=NonNegativeReals)

# Define the deviation variables
model.ds = Var(domain=NonNegativeReals)
model.dsf = Var(domain=NonNegativeReals)
model.dsp = Var(domain=NonNegativeReals)
model.dh = Var(domain=NonNegativeReals)
model.dhf = Var(domain=NonNegativeReals)
model.dhp = Var(domain=NonNegativeReals)
model.do = Var(domain=NonNegativeReals)
model.dof = Var(domain=NonNegativeReals)
model.dop = Var(domain=NonNegativeReals)

# Define the objective function
model.obj = Objective(expr = model.ds + model.dsf + model.dsp + model.dh + model.dhf + model.dhp + model.do + model.dof + model.dop, sense= minimize)

# Constraints

# Sales Constraints
model.sales1 = Constraint(expr = model.sf + model.sp == model.s)
model.sales2 = Constraint(expr = model.sf == 5*model.hf)
model.sales3 = Constraint(expr = model.sp == 2*model.hp)

# Hours Constraints
model.hours1 = Constraint(expr = model.hf + model.hp == model.h)
model.hours2 = Constraint(expr = model.hf <= 800)
model.hours3 = Constraint(expr = model.hp <= 320)

# Overtime Constraints
model.overtime1 = Constraint(expr = model.of + model.op == model.o)
model.overtime2 = Constraint(expr = model.of <= 100)

# Deviation Constraints
model.dev1 = Constraint(expr = model.ds - (model.s - 5500) >= 0)
model.dev2 = Constraint(expr = model.ds - (5500 - model.s) >= 0)
model.dev3 = Constraint(expr = model.dsf - (model.sf - 4000) >= 0)
model.dev4 = Constraint(expr = model.dsf - (4000 - model.sf) >= 0)
model.dev5 = Constraint(expr = model.dsp - (model.sp - 1500) >= 0)
model.dev6 = Constraint(expr = model.dsp - (1500 - model.sp) >= 0)
model.dev7 = Constraint(expr = model.dh - (model.h - 1120) >= 0)
model.dev8 = Constraint(expr = model.dh - (1120 - model.h) >= 0)
model.dev9 = Constraint(expr = model.dhf - (model.hf - 800) >= 0)
model.dev10 = Constraint(expr = model.dhf - (800 - model.hf) >= 0)
model.dev11 = Constraint(expr = model.dhp - (model.hp - 320) >= 0)
model.dev12 = Constraint(expr = model.dhp - (320 - model.hp) >= 0)
model.dev13 = Constraint(expr = model.do - (model.o - 150) >= 0)
model.dev14 = Constraint(expr = model.do - (150 - model.o) >= 0)
model.dev15 = Constraint(expr = model.dof - (model.of - 100) >= 0)
model.dev16 = Constraint(expr = model.dof - (100 - model.of) >= 0)
model.dev17 = Constraint(expr = model.dop - model.op >= 0)
model.dev18 = Constraint(expr = model.dop + model.op >= 0)

# Solve
SolverFactory('glpk').solve(model)

# Print the output
print('Amount of units sold:', model.s())
print('Amount of units sold by full-time employees:', model.sf())
print('Amount of units sold by part-time employees:', model.sp())
print('Total hours worked by all employees:', model.h())
print('Hours worked by full-time employees:', model.hf())
print('Hours worked by part-time employees:', model.hp())
print('Total overtime hours:', model.o())
print('Overtime hours by full-time employees:', model.of())
print('Overtime Hours by part-time employees:', model.op())
```

In this code, I have used sample values as 5500 for target sale units, 4000 for full-time employee target sales, 1500 for part-time employee target sales, 1120 for target hours, 800 for full time employee target hours, 320 for part time employee target hours, and 150 for target overtime hours.

### **Run the code Generated by GPT4**

In [ ]:

from pyomo.environ import *
from pyomo.opt import SolverFactory

# Create a model
model = ConcreteModel()

# Define the decision variables
model.s = Var(domain=NonNegativeReals)
model.sf = Var(domain=NonNegativeReals)
model.sp = Var(domain=NonNegativeReals)
model.h = Var(domain=NonNegativeReals)
model.hf = Var(domain=NonNegativeReals)
model.hp = Var(domain=NonNegativeReals)
model.o = Var(domain=NonNegativeReals)
model.of = Var(domain=NonNegativeReals)
model.op = Var(domain=NonNegativeReals)

# Define the deviation variables
model.ds = Var(domain=NonNegativeReals)
model.dsf = Var(domain=NonNegativeReals)
model.dsp = Var(domain=NonNegativeReals)
model.dh = Var(domain=NonNegativeReals)
model.dhf = Var(domain=NonNegativeReals)
model.dhp = Var(domain=NonNegativeReals)
model.do = Var(domain=NonNegativeReals)
model.dof = Var(domain=NonNegativeReals)
model.dop = Var(domain=NonNegativeReals)

# Define the objective function
model.obj = Objective(expr = model.ds + model.dsf + model.dsp + model.dh + model.dhf + model.dhp + model.do + model.dof + model.dop, sense= minimize)

# Constraints

# Sales Constraints
model.sales1 = Constraint(expr = model.sf + model.sp == model.s)
model.sales2 = Constraint(expr = model.sf == 5*model.hf)
model.sales3 = Constraint(expr = model.sp == 2*model.hp)

# Hours Constraints
model.hours1 = Constraint(expr = model.hf + model.hp == model.h)
model.hours2 = Constraint(expr = model.hf <= 800)
model.hours3 = Constraint(expr = model.hp <= 320)

# Overtime Constraints
model.overtime1 = Constraint(expr = model.of + model.op == model.o)
model.overtime2 = Constraint(expr = model.of <= 100)

# Deviation Constraints
model.dev1 = Constraint(expr = model.ds - (model.s - 5500) >= 0)
model.dev2 = Constraint(expr = model.ds - (5500 - model.s) >= 0)
model.dev3 = Constraint(expr = model.dsf - (model.sf - 4000) >= 0)
model.dev4 = Constraint(expr = model.dsf - (4000 - model.sf) >= 0)
model.dev5 = Constraint(expr = model.dsp - (model.sp - 1500) >= 0)
model.dev6 = Constraint(expr = model.dsp - (1500 - model.sp) >= 0)
model.dev7 = Constraint(expr = model.dh - (model.h - 1120) >= 0)
model.dev8 = Constraint(expr = model.dh - (1120 - model.h) >= 0)
model.dev9 = Constraint(expr = model.dhf - (model.hf - 800) >= 0)
model.dev10 = Constraint(expr = model.dhf - (800 - model.hf) >= 0)
model.dev11 = Constraint(expr = model.dhp - (model.hp - 320) >= 0)
model.dev12 = Constraint(expr = model.dhp - (320 - model.hp) >= 0)
model.dev13 = Constraint(expr = model.do - (model.o - 150) >= 0)
model.dev14 = Constraint(expr = model.do - (150 - model.o) >= 0)
model.dev15 = Constraint(expr = model.dof - (model.of - 100) >= 0)
model.dev16 = Constraint(expr = model.dof - (100 - model.of) >= 0)
model.dev17 = Constraint(expr = model.dop - model.op >= 0)
model.dev18 = Constraint(expr = model.dop + model.op >= 0)

# Solve
SolverFactory('glpk').solve(model)

# Print the output
print('Amount of units sold:', model.s())
print('Amount of units sold by full-time employees:', model.sf())
print('Amount of units sold by part-time employees:', model.sp())
print('Total hours worked by all employees:', model.h())
print('Hours worked by full-time employees:', model.hf())
print('Hours worked by part-time employees:', model.hp())
print('Total overtime hours:', model.o())
print('Overtime hours by full-time employees:', model.of())
print('Overtime Hours by part-time employees:', model.op())

Amount of units sold: 4640.0
Amount of units sold by full-time employees: 4000.0
Amount of units sold by part-time employees: 640.0
Total hours worked by all employees: 1120.0
Hours worked by full-time employees: 800.0
Hours worked by part-time employees: 320.0
Total overtime hours: 100.0
Overtime hours by full-time employees: 100.0
Overtime Hours by part-time employees: 0.0


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [ ]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

P1 = 4
P2 = 3
P3 = 2
P4 = 1

# Create a model
model = ConcreteModel()

# Define the decision variables
model.s = Var(domain=NonNegativeReals)
model.sf = Var(domain=NonNegativeReals)
model.sp = Var(domain=NonNegativeReals)
model.h = Var(domain=NonNegativeReals)
model.hf = Var(domain=NonNegativeReals)
model.hp = Var(domain=NonNegativeReals)
model.o = Var(domain=NonNegativeReals)
model.of = Var(domain=NonNegativeReals)
model.op = Var(domain=NonNegativeReals)

# Define the deviation variables
model.ds = Var(domain=NonNegativeReals)
model.dsf = Var(domain=NonNegativeReals)
model.dsp = Var(domain=NonNegativeReals)
model.dh = Var(domain=NonNegativeReals)
model.dhf = Var(domain=NonNegativeReals)
model.dhp = Var(domain=NonNegativeReals)
model.do = Var(domain=NonNegativeReals)
model.dof = Var(domain=NonNegativeReals)
model.dop = Var(domain=NonNegativeReals)

# Define the objective function
model.obj = Objective(expr = P1*model.ds + P2*model.dof + P3*(2*model.dhf + model.dhp) + P4*(model.do + 3*model.dop), sense=minimize)

# Constraints

# Sales Constraints
model.sales1 = Constraint(expr = model.sf + model.sp == model.s)
model.sales2 = Constraint(expr = model.sf == 5*model.hf)
model.sales3 = Constraint(expr = model.sp == 2*model.hp)

# Hours Constraints
model.hours1 = Constraint(expr = model.hf + model.hp == model.h)
model.hours2 = Constraint(expr = model.hf <= 800)
model.hours3 = Constraint(expr = model.hp <= 320)

# Overtime Constraints
model.overtime1 = Constraint(expr = model.of + model.op == model.o)
model.overtime2 = Constraint(expr = model.of <= 100)

# Deviation Constraints
model.dev1 = Constraint(expr = model.ds - (model.s - 5500) >= 0)
model.dev2 = Constraint(expr = model.ds - (5500 - model.s) >= 0)
model.dev3 = Constraint(expr = model.dsf - (model.sf - 4000) >= 0)
model.dev4 = Constraint(expr = model.dsf - (4000 - model.sf) >= 0)
model.dev5 = Constraint(expr = model.dsp - (model.sp - 1500) >= 0)
model.dev6 = Constraint(expr = model.dsp - (1500 - model.sp) >= 0)
model.dev7 = Constraint(expr = model.dh - (model.h - 1120) >= 0)
model.dev8 = Constraint(expr = model.dh - (1120 - model.h) >= 0)
model.dev9 = Constraint(expr = model.dhf - (model.hf - 800) >= 0)
model.dev10 = Constraint(expr = model.dhf - (800 - model.hf) >= 0)
model.dev11 = Constraint(expr = model.dhp - (model.hp - 320) >= 0)
model.dev12 = Constraint(expr = model.dhp - (320 - model.hp) >= 0)
model.dev13 = Constraint(expr = model.do - (model.o - 150) >= 0)
model.dev14 = Constraint(expr = model.do - (150 - model.o) >= 0)
model.dev15 = Constraint(expr = model.dof - (model.of - 100) >= 0)
model.dev16 = Constraint(expr = model.dof - (100 - model.of) >= 0)
model.dev17 = Constraint(expr = model.dop - model.op >= 0)
model.dev18 = Constraint(expr = model.dop + model.op >= 0)

# Solve
SolverFactory('glpk').solve(model)

# Print the output
print('Amount of units sold:', model.s())
print('Amount of units sold by full-time employees:', model.sf())
print('Amount of units sold by part-time employees:', model.sp())
print('Total hours worked by all employees:', model.h())
print('Hours worked by full-time employees:', model.hf())
print('Hours worked by part-time employees:', model.hp())
print('Total overtime hours:', model.o())
print('Overtime hours by full-time employees:', model.of())
print('Overtime Hours by part-time employees:', model.op())

Amount of units sold: 4640.0
Amount of units sold by full-time employees: 4000.0
Amount of units sold by part-time employees: 640.0
Total hours worked by all employees: 1120.0
Hours worked by full-time employees: 800.0
Hours worked by part-time employees: 320.0
Total overtime hours: 100.0
Overtime hours by full-time employees: 100.0
Overtime Hours by part-time employees: 0.0
